# Patient Data Query

This notebook queries patient data from multiple databases for a given prontuario.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_embrioes_congelados
- **Finops Silver Layer** (huntington_data_lake.duckdb):
  - silver.mesclada_vendas


In [17]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Input: Patient Prontuario


In [18]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Database Connections


In [19]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
        if not df.empty:
            display(df)
        else:
            print("No records found.")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [20]:
def query_biopsy_pgta_timeline(conn, prontuario):
    """Query biopsy and PGT-A timeline data for a specific prontuario"""
    print(f"\n=== BIOPSY PGT-A TIMELINE ===")
    query = f"""
    SELECT * FROM gold.biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in biopsy_pgta_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_billing_timeline(conn, prontuario):
    """Query billing timeline data for a specific prontuario"""
    print(f"\n=== BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in billing_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [21]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_mesclada_vendas(conn, prontuario):
    """Query silver.mesclada_vendas for a specific prontuario"""
    print(f"\n=== SILVER.MESCLADA_VENDAS ===")
    query = f"""
    SELECT * FROM silver.mesclada_vendas 
    WHERE prontuario = {prontuario}
    ORDER BY "DT Emissao" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.mesclada_vendas")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [22]:
def query_comprehensive_timeline(conn, prontuario):
    """Query comprehensive timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE BIOPSY PGT-A TIMELINE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_biopsy_pgta_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [23]:
def query_resumed_timeline(conn, prontuario):
    """Query resumed timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED BIOPSY PGT-A TIMELINE TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_biopsy_pgta_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [24]:
def query_embryoscope_timeline(conn, prontuario):
    """Query embryoscope timeline data for a specific prontuario"""
    print(f"\n=== EMBRYOSCOPE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryoscope_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryoscope_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_resumed_embryoscope_timeline(conn, prontuario):
    """Query resumed embryoscope timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED EMBRYOSCOPE TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_embryoscope_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_embryoscope_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_embryo_treatment (conn, prontuario):
    """Query embryoscope treatment data for a specific prontuario"""
    print(f"\n=== EMBRYOSCOPE TREATMENT ===")
    query = f"""
    SELECT 
        patient_PatientIDx
        ,	patient_PatientID
        ,	patient_FirstName	
        , patient_LastName	
        , patient_unit_huntington	
        , embryo_KIDDate
        , embryo_FertilizationTime 
        , treatment_TreatmentName	
        , patient_unit_huntington 
    FROM gold.embryoscope_embrioes 
    WHERE patient_PatientID = {prontuario}
    ORDER BY treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryoscope_embrioes")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_embryo_freeze_timeline(conn, prontuario):
    """Query embryo freeze timeline data for a specific prontuario"""
    print(f"\n=== EMBRYO FREEZE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryo_freeze_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryo_freeze_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_embryo_billing_timeline(conn, prontuario):
    """Query embryo billing timeline data for a specific prontuario"""
    print(f"\n=== EMBRYO BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryo_billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryo_billing_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_comprehensive_embryo_timeline(conn, prontuario):
    """Query comprehensive embryo timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE EMBRYO FREEZE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_embryo_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_embryo_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_cryopreservation_events_timeline(conn, prontuario):
    """Query cryopreservation events timeline data for a specific prontuario"""
    print(f"\n=== CRYOPRESERVATION EVENTS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.cryopreservation_events_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in cryopreservation_events_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_consultas_timeline(conn, prontuario):
    """Query consultation timeline data for a specific prontuario"""
    print(f"\n=== CONSULTAS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.consultas_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in consultas_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df

def query_resumed_consultas_timeline(conn, prontuario):
    """Query resumed consultation timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED CONSULTAS TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_consultas_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_consultas_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


def get_finops_data(conn, prontuario):
    """Get finops data for a specific patient"""
    print(f"\n=== RESUMED TRATAMENTOS TIMELINE (MOST RECENT RECORD) ===")

    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {prontuario}
    """

    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_consultas_timeline")
    if not df.empty:
        display(df)
    else:
        print("No records found.")
    return df


In [25]:
# Updated query function with biopsy PGT-A timeline and cryopreservation data
def query_patient_data_with_timeline(prontuario):
    """
    Query all patient data including timeline data from gold layer tables.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results including timeline data
    """
    print(f"\n{'='*80}")
    print(f"QUERYING DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_embrioes = query_embrioes_congelados(clinisys_con, prontuario)
        
        # Query mesclada table
        df_mesclada = query_mesclada_vendas(huntington_con, prontuario)

        # query Tratamentos timeline
        df_tratamentos_timeline = get_finops_data(huntington_con, prontuario)
        
        # Query biopsy PGT-A timeline
        df_biopsy_pgta = query_biopsy_pgta_timeline(huntington_con, prontuario)
        
        # Query billing timeline
        df_billing = query_billing_timeline(huntington_con, prontuario)
        
        # Query comprehensive timeline
        df_comprehensive = query_comprehensive_timeline(huntington_con, prontuario)
        
        # Query resumed timeline (most recent record only)
        df_resumed = query_resumed_timeline(huntington_con, prontuario)

        # Query embryoscope timeline
        df_embryoscope_timeline = query_embryoscope_timeline(huntington_con, prontuario)
        
        # Query resumed embryoscope timeline (most recent record only)
        df_resumed_embryoscope = query_resumed_embryoscope_timeline(huntington_con, prontuario)

        # Query embryoscope usage
        df_embryoscope = query_embryo_treatment(huntington_con, prontuario)
        
        # Query cryopreservation timeline data
        df_embryo_freeze_timeline = query_embryo_freeze_timeline(huntington_con, prontuario)
        df_embryo_billing_timeline = query_embryo_billing_timeline(huntington_con, prontuario)
        df_comprehensive_embryo_timeline = query_comprehensive_embryo_timeline(huntington_con, prontuario)
        df_cryopreservation_events_timeline = query_cryopreservation_events_timeline(huntington_con, prontuario)
        
        # Query consultation timeline data
        df_consultas_timeline = query_consultas_timeline(huntington_con, prontuario)
        df_resumed_consultas_timeline = query_resumed_consultas_timeline(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes)} records")
        print(f"- silver.mesclada_vendas: {len(df_mesclada)} records")
        print(f"- tratamentos_timeline: {len(df_tratamentos_timeline)} records")
        print(f"- biopsy_pgta_timeline: {len(df_biopsy_pgta)} records")
        print(f"- billing_timeline: {len(df_billing)} records")
        print(f"- comprehensive_timeline: {len(df_comprehensive)} records")
        print(f"- resumed_timeline: {len(df_resumed)} records")
        print(f"- embryoscope_timeline: {len(df_embryoscope_timeline)} records")
        print(f"- resumed_embryoscope_timeline: {len(df_resumed_embryoscope)} records")
        print(f"- embryoscope_treatment: {len(df_embryoscope)} records")
        print(f"- embryo_freeze_timeline: {len(df_embryo_freeze_timeline)} records")
        print(f"- embryo_billing_timeline: {len(df_embryo_billing_timeline)} records")
        print(f"- comprehensive_embryo_timeline: {len(df_comprehensive_embryo_timeline)} records")
        print(f"- cryopreservation_events_timeline: {len(df_cryopreservation_events_timeline)} records")
        print(f"- consultas_timeline: {len(df_consultas_timeline)} records")
        print(f"- resumed_consultas_timeline: {len(df_resumed_consultas_timeline)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'embrioes_congelados': df_embrioes,
            'mesclada_vendas': df_mesclada,
            'tratamentos_timeline': df_tratamentos_timeline,
            'biopsy_pgta_timeline': df_biopsy_pgta,
            'billing_timeline': df_billing,
            'comprehensive_timeline': df_comprehensive,
            'resumed_timeline': df_resumed,
            'embryoscope_timeline': df_embryoscope_timeline,
            'resumed_embryoscope_timeline': df_resumed_embryoscope,
            'embryoscope_treatment': df_embryoscope,
            'embryo_freeze_timeline': df_embryo_freeze_timeline,
            'embryo_billing_timeline': df_embryo_billing_timeline,
            'comprehensive_embryo_timeline': df_comprehensive_embryo_timeline,
            'cryopreservation_events_timeline': df_cryopreservation_events_timeline,
            'consultas_timeline': df_consultas_timeline,
            'resumed_consultas_timeline': df_resumed_consultas_timeline
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Query Finops Silver Layer Table


In [30]:
# Example usage with comprehensive timeline data including cryopreservation
# prontuario = '825_890'  # Patient with PGT-A data
prontuario = '749_579' # 36 biopsies (3 paid) - 28 pgta (16 paid)
# prontuario = '150_059' # 25 biopsies (3 paid) - 19 pgta (0 paid)

# prontuario = '158_696' # 40 biopsies (10 paid) - 17 pgta (6 paid)
# prontuario = '896_947' # 20 biopsies (0 paid) - 20 pgta (0 paid)
# prontuario = '882_954' # 21 biopsies (1 paid) - 21 pgta (3 paid)
# prontuario = '879_286' # 17 biopsies (1 paid) - 17 pgta (4 paid)
# prontuario = '809_017' # 22 biopsies (7 paid) - 22 pgta (4 paid)
# prontuario = '825_263' # 19 biopsies (1 paid) - 15 pgta (0 paid)
# prontuario = '511_257' # 30 biopsies (25 paid) - 25 pgta (8 paid)
# prontuario = '221_379' # 22 biopsies (1 paid) - 10 pgta (0 paid)
# prontuario = '839_259' # 21 biopsies (10 paid) - 10 pgta (10 paid)
# prontuario = '794_640' # 16 biopsies (13 paid) - 13 pgta (6 paid)
# prontuario = '872_534' # 23 biopsies (4 paid) - 6 pgta (6 paid)
# prontuario = '666_290' # 22 biopsies (8 paid) - 19 pgta (8 paid)
# prontuario = '153_411' # 15 biopsies (8 paid) - 13 pgta (8 paid)
# prontuario = '813_209' # 17 biopsies (15 paid) - 11 pgta (19 paid)
# prontuario = '826_457' # 18 biopsies (1 paid) - 18 pgta (21 paid)
# prontuario = '835_937' # 35 biopsies (21 paid) - 35 pgta (25 paid)
# prontuario = '851_077' # 17 biopsies (2 paid) - 17 pgta (10 paid)
# prontuario = '804_527' # 16 biopsies (1 paid) - 16 pgta (5 paid)
# prontuario = '877_692' # 16 biopsies (10 paid) - 10 pgta (5 paid)
# prontuario = '884_227' # 13 biopsies (0 paid) - 13 pgta (0 paid)
# prontuario = '868_915' # 13 biopsies (13 paid) - 13 pgta (4 paid)
# prontuario = '897_679' # 16 biopsies (0 paid) - 9 pgta (0 paid)
# prontuario = '813_112' # 24 biopsies (1 paid) - 2 pgta (0 paid)
# prontuario = '792_259' # 19 biopsies (6 paid) - 18 pgta (6 paid)
# prontuario = '183_278' # 12 biopsies (2 paid) - 12 pgta (2 paid)
# prontuario = '832_016' # 15 biopsies (2 paid) - 15 pgta (15 paid)
# prontuario = '667_294' # 12 biopsies (1 paid) - 12 pgta (5 paid)
# prontuario = '830_663' # 15 biopsies (1 paid) - 15 pgta (5 paid)

prontuario = '788592 '


# Query all patient data including comprehensive timeline and cryopreservation data
results_with_timeline = query_patient_data_with_timeline(prontuario)

# Access comprehensive timeline data
comprehensive_timeline = results_with_timeline['comprehensive_timeline']
if not comprehensive_timeline.empty:
    print(f"\nComprehensive Timeline for prontuario {prontuario}:")
    print(f"Total biopsies: {comprehensive_timeline['biopsy_count'].sum()}")
    print(f"Total biopsy payments: {comprehensive_timeline['biopsy_payment_count'].sum()}")
    print(f"Biopsy missing payments: {comprehensive_timeline['biopsy_missing_payment'].iloc[0]}")
    print(f"Total PGT-A tests: {comprehensive_timeline['pgta_test_count'].sum()}")
    print(f"Total PGT-A payments: {comprehensive_timeline['pgta_payment_count'].sum()}")
    print(f"PGT-A missing payments: {comprehensive_timeline['pgta_missing_payment'].iloc[0]}")
else:
    print(f"No comprehensive timeline data found for prontuario {prontuario}")

# Access cryopreservation timeline data
cryopreservation_events = results_with_timeline['cryopreservation_events_timeline']
if not cryopreservation_events.empty:
    print(f"\nCryopreservation Events Timeline for prontuario {prontuario}:")
    print(f"Total freezing events: {cryopreservation_events['freezing_events_count'].sum()}")
    print(f"Total billing events: {cryopreservation_events['billing_events_count'].sum()}")
    print(f"Total billing amount: R$ {cryopreservation_events['total_billing_amount'].sum():,.2f}")
    print(f"Current events missing payment: {cryopreservation_events['events_missing_payment'].iloc[0]}")
else:
    print(f"No cryopreservation events timeline data found for prontuario {prontuario}")

# Access embryo freeze timeline data
embryo_freeze_timeline = results_with_timeline['embryo_freeze_timeline']
if not embryo_freeze_timeline.empty:
    print(f"\nEmbryo Freeze Timeline for prontuario {prontuario}:")
    print(f"Total embryos frozen: {embryo_freeze_timeline['embryos_frozen_count'].sum()}")
    print(f"Total embryos unfrozen: {embryo_freeze_timeline['embryos_unfrozen_count'].sum()}")
    print(f"Total embryos discarded: {embryo_freeze_timeline['embryos_discarded_count'].sum()}")
    print(f"Current storage balance: {embryo_freeze_timeline['embryos_storage_balance'].iloc[0]}")
else:
    print(f"No embryo freeze timeline data found for prontuario {prontuario}")



QUERYING DATA FOR PRONTUARIO: 788592 
Query executed at: 2025-10-14 12:50:09.456405

=== VIEW_MICROMANIPULACAO ===
Found 4 records in view_micromanipulacao


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,21557,2403/24,788592,37,38,2024-07-24,None,16:35,17:05,13:35,74,3652,None,Não,None,None,None,16:25,08:00,None,08:00,08:00,08:00,08:00,20,24,None,None,None,None,None,Não,None,1,None,1,2024-07-24,ICSI,17 espermatozoides móveis progressivos\r\n,17 espermatozoides normais\r\n,None,None,None,None,None,None,0000000001,<NA>,<NA>,<NA>,3652,4005,None,4005,


=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 75 records in view_micromanipulacao_oocitos


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,114192,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,72a223e86481da8eaadcbb25225adfc0,2025-07-21 21:40:02,0,<NA>
1,114193,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,fc5a7eb8dc45638ebf9bc0be071bb9de,2025-07-21 21:40:02,0,<NA>
2,114194,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,a2925a5c09bfe27b1d1b973c446fc273,2025-07-21 21:40:02,0,<NA>
3,114195,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,538c6038f0843ccb6029abfeea16379d,2025-07-21 21:40:02,0,<NA>
4,114196,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,c7735850ebe5c558e4f6299bd2ea8d0a,2025-07-21 21:40:02,0,<NA>
5,114197,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,e0747ecb69a3007fcc518d5f399e08fc,2025-07-21 21:40:02,0,<NA>
6,114198,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,a050f0b3baf31ec522c2cf99ced0dead,2025-07-21 21:40:02,0,<NA>
7,114199,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,883690f272ff330f2c5f8ff24decf216,2025-07-21 21:40:02,0,<NA>
8,114200,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,c90dddc13eaa130e6435671d9e0006f6,2025-07-21 21:40:02,0,<NA>
9,114201,12889,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4f77738a6d5826fa64a7b0ae19acfecd,2025-07-21 21:40:02,0,<NA>



=== VIEW_TRATAMENTOS ===
Found 1 records in view_tratamentos


...


=== CONSULTAS TIMELINE ===
Found 2 records in consultas_timeline


,prontuario,period_month,consultas_events_count,cumulative_consultas_events,billing_events_count,total_billing_amount,total_quantity,cumulative_billing_events,cumulative_billing_amount,cumulative_quantity,consultas_missing_payment
0,788592.0,2024-10,1,2.0,0,0.0,0.0,0.0,0.0,0.0,2.0
1,788592.0,2024-04,1,1.0,0,0.0,0.0,0.0,0.0,0.0,1.0



=== RESUMED CONSULTAS TIMELINE (MOST RECENT RECORD) ===
Found 1 records in resumed_consultas_timeline


,prontuario,period_month,consultas_events_count,cumulative_consultas_events,billing_events_count,total_billing_amount,total_quantity,cumulative_billing_events,cumulative_billing_amount,cumulative_quantity,consultas_missing_payment
0,788592.0,2024-10,1,2.0,0,0.0,0.0,0.0,0.0,0.0,2.0



QUERY SUMMARY FOR PRONTUARIO 788592 
- view_micromanipulacao: 4 records
- view_micromanipulacao_oocitos: 75 records
- view_tratamentos: 1 records
- view_embrioes_congelados: 35 records
- silver.mesclada_vendas: 50 records
- tratamentos_timeline: 1 records
- biopsy_pgta_timeline: 3 records
- billing_timeline: 1 records
- comprehensive_timeline: 4 records
- resumed_timeline: 1 records
- embryoscope_timeline: 1 records
- resumed_embryoscope_timeline: 1 records
- embryoscope_treatment: 17 records
- embryo_freeze_timeline: 7 records
- embryo_billing_timeline: 21 records
- comprehensive_embryo_timeline: 33 records
- cryopreservation_events_timeline: 8 records
- consultas_timeline: 2 records
- resumed_consultas_timeline: 1 records

Database connections closed.

Comprehensive Timeline for prontuario 788592 :
Total biopsies: 29
Total biopsy payments: 1
Biopsy missing payments: 22.0
Total PGT-A tests: 29
Total PGT-A payments: 0
PGT-A missing payments: 29.0

Cryopreservation Events Timeline for 

In [31]:
# Example: Query multiple prontuarios
# prontuarios = ['160751', '171805', '876950']
# all_results = {}
# 
# for pront in prontuarios:
#     print(f"\nQuerying prontuario: {pront}")
#     all_results[pront] = query_patient_data(pront)
# 
# # Compare results
# for pront, results in all_results.items():
#     print(f"Prontuario {pront}: {len(results['tratamentos'])} tratamentos")
